In [15]:
import torch as t
import math
import numpy as np

FRAME_TIME = 0.1  # time interval
GRAVITY_ACCEL = 0.12  # gravity constant
BOOST_ACCEL = 0.18  # thrust constant
ROTATION_ACCEL = np.radians(20)  # rotation constant

action = t.tensor([0.5, 1.])
state = t.tensor([0.7854, 0., 0., 0., 0., 0.])

delta_state_ori = ROTATION_ACCEL* FRAME_TIME*t.tensor([0., 1, 0., 0., 0., 0.])

sin = t.matmul(t.tensor([[0., 0., 0., 0., 0., 0.],
                         [0., 0., 0., 0., 0., 0.],
                         [0., 0., 0., 0., 0., 0.],
                         [-1., 0., 0., 0., 0., 0.],
                         [0., 0., 0., 0., 0., 0.],
                         [0., 0., 0., 0., 0., 0.]]),t.sin(state))
cos = t.matmul(t.tensor([[0., 0., 0., 0., 0., 0.],
                         [0., 0., 0., 0., 0., 0.],
                         [0., 0., 0., 0., 0., 0.],
                         [0., 0., 0., 0., 0., 0.],
                         [0., 0., 0., 0., 0., 0.],
                         [1., 0., 0., 0., 0., 0.]]),t.cos(state))
print(sin)
print(cos)
cos_sin = cos+sin
delta_state_pos = BOOST_ACCEL* FRAME_TIME*t.tensor([0., 0., 0., 1., 0., 1.])*cos_sin # x and y components of acceleration

delta_state_combine = delta_state_ori + delta_state_pos

action_mat = t.tensor([[0., 0.],
                 [2., 0.],
                 [0., 0.],
                 [0., 1.],
                 [0., 0.],
                 [0., 1.]])
a_a = t.matmul(action_mat, action) + t.tensor([0., -1., 0., 0., 0., 0.]) # get ori and pos accelerations
print(a_a)

delta_state = delta_state_combine*a_a

state_gravity = GRAVITY_ACCEL * FRAME_TIME*t.tensor([0., 0., 0., 0., 0., -1.])
state = state + delta_state + state_gravity

step_mat = t.tensor([[1., FRAME_TIME, 0., 0., 0., 0.],
                     [0., 1., 0., 0., 0., 0.],
                     [0., 0., 1., FRAME_TIME, 0., 0.],
                     [0., 0., 0., 1., 0., 0.],
                     [0., 0., 0., 0., 1., FRAME_TIME,],
                     [0., 0., 0., 0., 0., 1.]])
state = t.matmul(step_mat, state)

print(state)


tensor([ 0.0000,  0.0000,  0.0000, -0.7071,  0.0000,  0.0000])
tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.7071])
tensor([0., 0., 0., 1., 0., 1.])
tensor([ 7.8540e-01,  0.0000e+00, -1.2728e-03, -1.2728e-02,  7.2790e-05,
         7.2790e-04])


In [29]:
import numpy as np
import time
import torch as t
import random
import math

N=10

states=t.zeros(N,1, 4)

for i in range(N):
    state = [i,i,i,i]  # vertical state
    state = t.tensor(state, requires_grad=False).float()
    states[i, :, :] = state
print(states[2,:,:])
print(t.squeeze(states[2,:,:]))

tensor([[2., 2., 2., 2.]])
tensor([2., 2., 2., 2.])


In [13]:
import torch as t
import numpy as np
import math
import random

N=5
initial = np.zeros((N,6))
for i in range(N):
    initial[i] = [math.pi*(random.random()-0.5), 0.1*math.pi*(random.random()-0.5), # orientation state
             (random.random()-0.5), 0.1*(random.random()-0.5), # horizontal state
             0.5*(random.random()+1), -0.1*random.random()]  # vertical state

print(initial)
state = t.from_numpy(initial)
print(state)

[[-1.45254147 -0.1201659  -0.31123233  0.03281846  0.79113479 -0.00707725]
 [-0.89370754 -0.12677698  0.29161829  0.00910432  0.59386505 -0.01517993]
 [ 1.40664903  0.1164549   0.03859754 -0.02730336  0.82349343 -0.05121958]
 [ 1.3307975   0.00624109  0.40385437 -0.02536792  0.65310533 -0.0442462 ]
 [-0.68686895  0.13296215 -0.02277104 -0.02018893  0.83103639 -0.02333656]]
tensor([[-1.4525, -0.1202, -0.3112,  0.0328,  0.7911, -0.0071],
        [-0.8937, -0.1268,  0.2916,  0.0091,  0.5939, -0.0152],
        [ 1.4066,  0.1165,  0.0386, -0.0273,  0.8235, -0.0512],
        [ 1.3308,  0.0062,  0.4039, -0.0254,  0.6531, -0.0442],
        [-0.6869,  0.1330, -0.0228, -0.0202,  0.8310, -0.0233]],
       dtype=torch.float64)
